[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/francisco-ortin/data-science-course/blob/main/statistics/confidence_intervals.ipynb)
[![License: CC BY-NC-SA 4.0](https://img.shields.io/badge/License-CC%20BY--NC--SA%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

# Confidence intervals

Confidence intervals provide a range of plausible values for an unknown parameter (e.g., mean), given a confidence level (e.g., 95%). The confidence level represents the probability that the interval contains the true value of the parameter. In this notebook, we will see how to compute confidence intervals from a data sample, using two different approaches.

In [1]:
# make sure the required packages are installed
%pip install numpy matplotlib scipy --quiet
# if running in colab, install the required packages and copy the necessary files
directory='data-science-course/statistics'
if get_ipython().__class__.__module__.startswith('google.colab'):
    !git clone --depth 1 https://github.com/francisco-ortin/data-science-course.git  2>/dev/null
    !cp --update {directory}/*.py .
    !mkdir -p img data
    !cp {directory}/img/* img/.
    !cp {directory}/data/* data/.

Note: you may need to restart the kernel to use updated packages.


## Dataset

Let's create a random dataset.

In [2]:
import numpy as np
random_seed = 42
np.random.seed(random_seed)
sample_size = 10000
dataset = np.random.rand(sample_size, 6)  # 1000 samples, 6 variables (features or columns); the last column is the target variable
print(f"Shape of the dataset: {dataset.shape}.")

Shape of the dataset: (10000, 6).


## Getting the accuracies of retrained models

Let's create two models for predicting continuous numeric values (float numbers).

In [3]:
from models import Model
modelA = Model(distribution='normal', mean=0.5, std=0.1)
modelB = Model(distribution='normal', mean=0.8, std=0.05)

In a loop, we take one sample to train the model and evaluate it with the remaining samples. All the accuracies are stored in a list.

In [4]:
iterations = 30
modelA_accuracies, modelB_accuracies = [], []
for i in range(iterations):
    np.random.shuffle(dataset)
    train_dataset = dataset[:-iterations, :]
    test_dataset = dataset[-iterations:, :]
    modelA.train(train_dataset)
    modelB.train(train_dataset)
    modelA_accuracies.append(modelA.accuracy(test_dataset))
    modelB_accuracies.append(modelB.accuracy(test_dataset))
print(f"Model A accuracy mean: {np.mean(modelA_accuracies):.4f}.")
print(f"Model B accuracy mean: {np.mean(modelB_accuracies):.4f}.")
from utils import five_number_summary
print("Summary of accuracies for model A:")
five_number_summary(modelA_accuracies, show=True)
print("Summary of accuracies for model B:")
five_number_summary(modelB_accuracies, show=True)
pass

Model A accuracy mean: 0.5024.
Model B accuracy mean: 0.7954.
Summary of accuracies for model A:
	Minimum: 0.2548168443870841.
	1st quartile (Q1): 0.4154823624088747.
	Median (Q2): 0.49231586820109824.
	3rd quartile (Q3): 0.5748866525007461.
	Maximum: 0.7065539181426641.
Summary of accuracies for model B:
	Minimum: 0.7120258582207911.
	1st quartile (Q1): 0.7528029611084828.
	Median (Q2): 0.7949051177340176.
	3rd quartile (Q3): 0.8399295403424012.
	Maximum: 0.8773435595915652.


Let's test for normality both lists of accuracies.

In [5]:
from scipy.stats import shapiro
_, p_value_A = shapiro(modelA_accuracies)
_, p_value_B = shapiro(modelB_accuracies)
print(f"Do the model A accuracies follow a normal distribution? {p_value_A > 0.05}.")
print(f"Do the model B accuracies follow a normal distribution? {p_value_B > 0.05}.")


Do the model A accuracies follow a normal distribution? True.
Do the model B accuracies follow a normal distribution? True.


## ✨ Questions ✨

1. Can we say that model B will perform better than A?
2. Why?

### Answers

*Write your answers here.*



## Confidence intervals for normal distribution samples

We can compute the confidence interval for a normal distribution sample using either a normal or students' t-distribution

In [6]:
import utils
ci_model_a = utils.confidence_interval(sample=modelA_accuracies, confidence_level=0.95)
ci_model_b = utils.confidence_interval(sample=modelB_accuracies, confidence_level=0.95)

print (f"Confidence interval for model A: {ci_model_a}.")
print (f"Confidence interval for model B: {ci_model_b}.")
# Do confidence intervals overlap?
overlap = utils.do_intervals_overlap(ci_model_a, ci_model_b)
print(f"Confidence intervals overlap? {overlap}.")
print(f"We can{'not' if overlap else ''} say that there are significant differences between the models.")
if not overlap:
    better_model = 'A' if ci_model_a[0] > ci_model_b[0] else 'B'
    print(f"Model {better_model} has significantly higher accuracy.")

Confidence interval for model A: (0.45897238972429033, 0.5458824793282182).
Confidence interval for model B: (0.7763983165655681, 0.8143048820845595).
Confidence intervals overlap? False.
We can say that there are significant differences between the models.
Model B has significantly higher accuracy.


## Confidence intervals for non-normal distribution samples, using bootstrapping

If the sample does not follow a normal distribution, we can use bootstrapping to compute the confidence intervals.

In [7]:
bootstrap_iterations = 10_000
bootstrap_sample_size = sample_size 
modelA_accuracies_bootstrap, modelB_accuracies_bootstrap = [], []
# bootstrap resampling
for i in range(bootstrap_iterations):
    # create a list of indices to sample from the dataset with replacement
    indices = np.random.choice(sample_size, bootstrap_sample_size, replace=True)
    bootstrap_sample = dataset[indices, :]
    modelA_accuracies_bootstrap.append(modelA.accuracy(bootstrap_sample))
    modelB_accuracies_bootstrap.append(modelB.accuracy(bootstrap_sample))
# compute the bootstrapped estimate of the mean and confidence intervals (percentile method) 
print(f"Bootstrapped mean for model A: {np.mean(modelA_accuracies_bootstrap):.4f}.")
print(f"Bootstrapped mean for model B: {np.mean(modelB_accuracies_bootstrap):.4f}.")    

Bootstrapped mean for model A: 0.5001.
Bootstrapped mean for model B: 0.7991.


In [8]:
bootstrap_ci_model_a = (np.percentile(modelA_accuracies_bootstrap, 2.5), np.percentile(modelA_accuracies_bootstrap, 97.5))
bootstrap_ci_model_b = (np.percentile(modelB_accuracies_bootstrap, 2.5), np.percentile(modelB_accuracies_bootstrap, 97.5))
print (f"Bootstrapped confidence interval for model A: {bootstrap_ci_model_a}.")
print (f"Bootstrapped confidence interval for model B: {bootstrap_ci_model_b}.")
# Do confidence intervals overlap?
overlap = utils.do_intervals_overlap(bootstrap_ci_model_a, bootstrap_ci_model_b)
print(f"Bootstrapped confidence intervals overlap? {overlap}.")
print(f"We can{'not' if overlap else ''} say that there are significant differences between the models.")
if not overlap:
    better_model = 'A' if bootstrap_ci_model_a[0] > bootstrap_ci_model_b[0] else 'B'
    print(f"Model {better_model} has significantly higher accuracy.")

Bootstrapped confidence interval for model A: (0.30564263141144016, 0.6924568639198916).
Bootstrapped confidence interval for model B: (0.6998564158900382, 0.8987474719736712).
Bootstrapped confidence intervals overlap? False.
We can say that there are significant differences between the models.
Model B has significantly higher accuracy.
